<a href="https://colab.research.google.com/github/lowbee2019/Algorithm/blob/master/%E6%89%8B%E5%86%99MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls -al

total 16
drwxr-xr-x 1 root root 4096 Oct 14 16:31 .
drwxr-xr-x 1 root root 4096 Oct 27 02:54 ..
drwxr-xr-x 1 root root 4096 Oct 23 16:14 .config
drwxr-xr-x 1 root root 4096 Oct 14 16:31 sample_data


In [2]:
!mkdir data

In [4]:
import torch
from torchvision import datasets,transforms
from torch import nn,optim
import torch.nn.functional as F

In [7]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,),(0.3081,))
])

In [8]:
traindata = datasets.MNIST('data',train=True,download=True,transform=transform)
testdata = datasets.MNIST('data',train=False,download=True,transform=transform)

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [9]:
!ls data

MNIST


In [10]:
!ls -la

total 20
drwxr-xr-x 1 root root 4096 Oct 27 09:56 .
drwxr-xr-x 1 root root 4096 Oct 27 09:31 ..
drwxr-xr-x 1 root root 4096 Oct 23 16:14 .config
drwxr-xr-x 3 root root 4096 Oct 27 09:56 data
drwxr-xr-x 1 root root 4096 Oct 14 16:31 sample_data


In [11]:
!ls data/MNIST/

processed  raw


In [12]:
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet,self).__init__()
    self.c1 = nn.Conv2d(1,6,(5,5))
    self.c3 = nn.Conv2d(6,16,(5,5))
    self.fc1 = nn.Linear(16*4*4,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)
  
  def forward(self,x):
    x = F.max_pool2d(F.relu(self.c1(x)),(2,2))
    x = F.max_pool2d(F.relu(self.c3(x)),(2,2))
    x = x.view(-1,self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
  
  def num_flat_features(self,x):
    size = x.size()[1:] #批处理导致维度上升一维，因此从第二位开始，如图片是4*2*2，则特征数量就是16
    num_features = 1
    for s in size:
      num_features *=s
    return num_features

In [13]:
CUDA = torch.cuda.is_available()

In [14]:
if CUDA:
  lenet = LeNet().cuda()
else:
  lenet = LeNet()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lenet.parameters(),lr=0.001,momentum=0.9)

In [15]:
lenet

LeNet(
  (c1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (c3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [16]:
trainloader = torch.utils.data.DataLoader(traindata,batch_size=4,shuffle=True,num_workers=2)

In [17]:
def train(model,criterion,optimizer,epochs):
  for e in range(epochs):
    running_loss = 0.
    for i,data in enumerate(trainloader,0):
      inputs,labels = data
      if CUDA:
        inputs,labels = inputs.cuda(),labels.cuda()
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs,labels)
      loss.backward()
      optimizer.step()

      running_loss +=loss.item()
      if i%1000 ==999:
        print('[Epoch:%d,Batch:%5d] Loss:%.3f'%(e+1,i+1,running_loss/1000))
        running_loss=0
  print("Finished Training")

In [18]:
train(lenet,criterion,optimizer,epochs=2)

[Epoch:1,Batch: 1000] Loss:1.159
[Epoch:1,Batch: 2000] Loss:0.342
[Epoch:1,Batch: 3000] Loss:0.249
[Epoch:1,Batch: 4000] Loss:0.173
[Epoch:1,Batch: 5000] Loss:0.155
[Epoch:1,Batch: 6000] Loss:0.137
[Epoch:1,Batch: 7000] Loss:0.139
[Epoch:1,Batch: 8000] Loss:0.123
[Epoch:1,Batch: 9000] Loss:0.110
[Epoch:1,Batch:10000] Loss:0.103
[Epoch:1,Batch:11000] Loss:0.108
[Epoch:1,Batch:12000] Loss:0.094
[Epoch:1,Batch:13000] Loss:0.102
[Epoch:1,Batch:14000] Loss:0.079
[Epoch:1,Batch:15000] Loss:0.071
[Epoch:2,Batch: 1000] Loss:0.067
[Epoch:2,Batch: 2000] Loss:0.072
[Epoch:2,Batch: 3000] Loss:0.062
[Epoch:2,Batch: 4000] Loss:0.079
[Epoch:2,Batch: 5000] Loss:0.060
[Epoch:2,Batch: 6000] Loss:0.068
[Epoch:2,Batch: 7000] Loss:0.058
[Epoch:2,Batch: 8000] Loss:0.071
[Epoch:2,Batch: 9000] Loss:0.066
[Epoch:2,Batch:10000] Loss:0.056
[Epoch:2,Batch:11000] Loss:0.058
[Epoch:2,Batch:12000] Loss:0.061
[Epoch:2,Batch:13000] Loss:0.063
[Epoch:2,Batch:14000] Loss:0.048
[Epoch:2,Batch:15000] Loss:0.047
Finished T

In [19]:
torch.save(lenet,'model.pkl')

In [20]:
!ls

data  model.pkl  sample_data


In [21]:
lenet = torch.load('model.pkl')

In [22]:
testloader = torch.utils.data.DataLoader(testdata,batch_size=4,shuffle=False,num_workers=2)

In [25]:
def test(testloader,model):
  correct = 0
  total = 0
  for data in testloader:
    images,labels = data
    if CUDA:
      images = images.cuda()
      labels = labels.cuda()
    outputs = model(images)
    _,predicted = torch.max(outputs.data,1)
    print(predicted)
    total +=labels.size(0)
    correct += (predicted ==labels).sum()
  print('Accuracy on the test set: %d %%'%(100.*correct/total))

In [26]:
#train(lenet,criterion,optimizer,epochs=2)
test(testloader,lenet)

tensor([7, 2, 1, 0], device='cuda:0')
tensor([4, 1, 4, 9], device='cuda:0')
tensor([5, 9, 0, 6], device='cuda:0')
tensor([9, 0, 1, 5], device='cuda:0')
tensor([9, 7, 3, 4], device='cuda:0')
tensor([9, 6, 6, 5], device='cuda:0')
tensor([4, 0, 7, 4], device='cuda:0')
tensor([0, 1, 3, 1], device='cuda:0')
tensor([3, 4, 7, 2], device='cuda:0')
tensor([7, 1, 2, 1], device='cuda:0')
tensor([1, 7, 4, 2], device='cuda:0')
tensor([3, 5, 1, 2], device='cuda:0')
tensor([4, 4, 6, 3], device='cuda:0')
tensor([5, 5, 6, 0], device='cuda:0')
tensor([4, 1, 9, 5], device='cuda:0')
tensor([7, 8, 9, 3], device='cuda:0')
tensor([7, 4, 6, 4], device='cuda:0')
tensor([3, 0, 7, 0], device='cuda:0')
tensor([2, 9, 1, 7], device='cuda:0')
tensor([3, 2, 9, 7], device='cuda:0')
tensor([7, 6, 2, 7], device='cuda:0')
tensor([8, 4, 7, 3], device='cuda:0')
tensor([6, 1, 3, 6], device='cuda:0')
tensor([9, 3, 1, 4], device='cuda:0')
tensor([1, 7, 6, 9], device='cuda:0')
tensor([6, 0, 5, 4], device='cuda:0')
tensor([9, 9